In [2]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

##### First step: Have GPT-4o-mini figure out which links are relevant
##### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [16]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/microsoft/VibeVoice-1.5B',
 '/xai-org/grok-2',
 '/openbmb/MiniCPM-V-4_5',
 '/Qwen/Qwen-Image-Edit',
 '/deepseek-ai/DeepSeek-V3.1',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/syncora/synthetic-generation',
 '/spaces/multimodalart/Qwen-Image-Edit-Fast',
 '/spaces/Qwen/Qwen-Image-Edit',
 '/spaces',
 '/datasets/syncora/developer-productivity-simulated-behavioral-data',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/syncora/synthetic-healthcare-admissions',
 '/datasets/openai/healthbench',
 '/datasets/nvidia/Nemotron-Post-Training-Dataset-v2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/m

In [17]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

##### Second step: Make the brochure

Assemble all the details into another gtp4-o prompt

In [9]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [10]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [11]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [22]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-1.5B\nUpdated\n3 days ago\n•\n67.6k\n•\n1.07k\nxai-org/grok-2\nUpdated\n7 days ago\n•\n3.99k\n•\n862\nopenbmb/MiniCPM-V-4_5\nUpdated\nabout 13 hours ago\n•\n8.46k\n•\n725\nQwen/Qwen-Image-Edit\nUpdated\n6 days ago\n•\n72.3k\n•\n1.53k\ndeepseek-ai/DeepSeek-V3.1\nUpdated\n4 days ago\n•\n72.4k\n•\n660\nBrowse 1M+ models\nSpaces\nRunning\n12.5k\n12.5k\nDeepSite v2\n🐳\nGener

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [19]:
get_brochure = create_brochure("HuggingFace", "https://huggingface.co")
get_brochure

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'forums', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Company Brochure

## Overview
**Hugging Face** is at the forefront of the artificial intelligence (AI) and machine learning (ML) revolution, providing a collaborative platform where developers and researchers can come together to build, share, and innovate. The company is widely recognized as a central hub for ML resources, hosting over 1 million models, 250k datasets, and numerous AI applications. Hugging Face is dedicated to creating an inclusive and open environment for the ML community, fostering innovation and collaboration for researchers, engineers, and data enthusiasts.

## Our Mission
*Hugging Face: The AI community building the future* — Our goal is to enhance collaboration in the ML field while making cutting-edge models and datasets more accessible to everyone, from individual developers to top enterprises.

## Customers
Over **50,000 organizations** utilize Hugging Face, including some of the largest names in tech like Microsoft, Google, and Amazon. We cater to a variety of markets, including non-profit organizations and large enterprises, ensuring everyone has access to machine learning tools and resources that promote growth and development.

## Community Engagement
Hugging Face offers a **strong support community**, facilitating collaboration and knowledge sharing. Users can explore trending models, participate in discussions, and contribute to an ever-growing library of resources. This community-driven approach has established Hugging Face as a pioneering influence in AI development.

## Company Culture
At Hugging Face, we embrace a culture of **open-source collaboration and innovation**. We believe in the power of collective intelligence and the importance of transparency in AI development. Our initiatives foster an encouraging and inclusive environment where contributions from individuals of diverse backgrounds are valued and celebrated.

- **Inclusivity:** Embracing diverse perspectives and backgrounds within our team and the broader community.
- **Collaboration:** Encouraging partnerships among peers to foster idea exchange and innovative results.
- **Transparency:** Committing to open dialogue and sharing developments with our community.

## Careers & Opportunities
We are continually looking for talented individuals to join our team and contribute to our mission. Whether you are an experienced software engineer, data scientist, researcher, or community manager, there are numerous opportunities to grow your career at Hugging Face:

- Work in a dynamic and inclusive environment.
- Participate in projects that push the boundaries of AI and ML.
- Collaborate with leading experts in the field and contribute to innovative research.

### Available Positions
Visit our [Jobs page](https://huggingface.co/jobs) to explore current openings!

## Join Us
Whether you are an enterprise looking to leverage AI capabilities, a developer eager to explore cutting-edge tools, or a potential recruit wanting to join a revolutionary team, **Hugging Face** welcomes you. Together, let’s build the future of artificial intelligence!

For more information, visit our website: [huggingface.co](https://huggingface.co)  
Follow us on social media for updates and community interactions:  
- [GitHub](https://github.com/huggingface)  
- [Twitter](https://twitter.com/huggingface)  
- [LinkedIn](https://www.linkedin.com/company/huggingface)  
- [Discord](https://discord.gg/huggingface)  

Let's shape the future of AI together!

"# Hugging Face Company Brochure\n\n## Overview\n**Hugging Face** is at the forefront of the artificial intelligence (AI) and machine learning (ML) revolution, providing a collaborative platform where developers and researchers can come together to build, share, and innovate. The company is widely recognized as a central hub for ML resources, hosting over 1 million models, 250k datasets, and numerous AI applications. Hugging Face is dedicated to creating an inclusive and open environment for the ML community, fostering innovation and collaboration for researchers, engineers, and data enthusiasts.\n\n## Our Mission\n*Hugging Face: The AI community building the future* — Our goal is to enhance collaboration in the ML field while making cutting-edge models and datasets more accessible to everyone, from individual developers to top enterprises.\n\n## Customers\nOver **50,000 organizations** utilize Hugging Face, including some of the largest names in tech like Microsoft, Google, and Amazon

### Brochure Translator to Bengali 

In [30]:
system_prompt = """
You are a Bengali translator who can translate English text to Bengali. You will be given a brochure of a company and you need to translate it into Bengali. Make sure 
to keep the formatting and layout of the original brochure intact. Any url present in the brochure should be in it's original text.
"""

In [26]:
def user_prompt_translate_brochure(company_name, brochure):
    user_prompt = f"You are given the brochure for the company {company_name}."
    user_prompt += f"Please translate the following brochure to Bengali: {brochure}"
    return user_prompt


In [27]:
user_prompt_translate_brochure("HuggingFace", get_brochure)

"You are given the brochure for the company HuggingFace.Please translate the following brochure to Bengali: # Hugging Face Company Brochure\n\n## Overview\n**Hugging Face** is at the forefront of the artificial intelligence (AI) and machine learning (ML) revolution, providing a collaborative platform where developers and researchers can come together to build, share, and innovate. The company is widely recognized as a central hub for ML resources, hosting over 1 million models, 250k datasets, and numerous AI applications. Hugging Face is dedicated to creating an inclusive and open environment for the ML community, fostering innovation and collaboration for researchers, engineers, and data enthusiasts.\n\n## Our Mission\n*Hugging Face: The AI community building the future* — Our goal is to enhance collaboration in the ML field while making cutting-edge models and datasets more accessible to everyone, from individual developers to top enterprises.\n\n## Customers\nOver **50,000 organizat

In [28]:
def display_bengali_brochure():
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_translate_brochure("HuggingFace", get_brochure)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result

In [31]:
display_bengali_brochure()

# Hugging Face কোম্পানির ব্রোশার

## সাধারণ তথ্য
**Hugging Face** কৃত্রিম বুদ্ধিমত্তা (AI) এবং মেশিন লার্নিং (ML) বিপ্লবের অগ্রভাগে রয়েছে, একটি সহযোগিতামূলক প্ল্যাটফর্ম প্রদান করে যেখানে ডেভেলপার এবং গবেষকরা একত্রে কাজ করতে, শেয়ার করতে এবং নতুনত্ব আনতে পারেন। কোম্পানিটি ML সম্পদের জন্য একটি কেন্দ্রীয় কেন্দ্র হিসেবে ব্যাপকভাবে পরিচিত, যেখানে ১ মিলিয়নেরও বেশি মডেল, ২৫০k ডেটাসেট এবং অসংখ্য AI অ্যাপ্লিকেশন রয়েছে। Hugging Face ML সম্প্রদায়ের জন্য একটি অন্তর্ভুক্তিমূলক এবং উন্মুক্ত পরিবেশ তৈরি করতে প্রতিশ্রুতিবদ্ধ, গবেষক, প্রকৌশলী এবং ডেটা উন্মাদদের জন্য নতুনত্ব এবং সহযোগিতা উত্সাহিত করে।

## আমাদের লক্ষ্য
*Hugging Face: AI সম্প্রদায় ভবিষ্যত গড়ছে* — আমাদের লক্ষ্য হল ML ক্ষেত্রে সহযোগিতা বৃদ্ধি করা এবং বিশেষ করে উন্নত মডেল এবং ডেটাসেটকে সকলের জন্য আরও সহজলভ্য করে তোলা, একক ডেভেলপার থেকে শীর্ষ এন্টারপ্রাইজ পর্যন্ত।

## গ্রাহকরা
**৫০,০০০ এরও বেশি সংগঠন** Hugging Face ব্যবহার করে, যার মধ্যে মাইক্রোসফ্ট, গুগল, এবং অ্যামাজনের মতো কিছু বৃহত্তম প্রযুক্তি নাম অন্তর্ভুক্ত রয়েছে। আমরা বিভিন্ন বাজারে সেবা প্রদান করি, যার মধ্যে অলাভজনক সংস্থা এবং বড় এন্টারপ্রাইজ রয়েছে, নিশ্চিত করছি যে সবাই মেশিন লার্নিং টুলস এবং সম্পদে প্রবেশাধিকার পায় যা বৃদ্ধ এবং উন্নয়নকে উত্সাহিত করে।

## সম্প্রদায়ের সংযুক্তি
Hugging Face একটি **শক্তিশালী সমর্থন সম্প্রদায়** প্রদান করে, সহযোগিতা এবং জ্ঞান ভাগাভাগির সুবিধা দেয়। ব্যবহারকারীরা জনপ্রিয় মডেলগুলি অন্বেষণ করতে, আলোচনা করতে অংশগ্রহণ করতে এবং একটি ক্রমবর্ধমান সম্পদ লাইব্রেরিতে অবদান রাখতে পারে। এই সম্প্রদায়-চালিত পন্থা Hugging Face কে AI উন্নয়নে একটি পথপ্রদর্শক প্রভাব হিসেবে প্রতিষ্ঠা করেছে।

## কোম্পানির সংস্কৃতি
Hugging Face-এ, আমরা **ওপেন-সোর্স সহযোগিতা এবং নতুনত্বের** সংস্কৃতিকে গ্রহণ করি। আমরা সম্মিলিত বুদ্ধির শক্তি এবং AI উন্নয়নের ক্ষেত্রে স্বচ্ছতার গুরুত্বে বিশ্বাস করি। আমাদের উদ্যোগগুলি একটি উৎসাহজনক এবং অন্তর্ভুক্তিমূলক পরিবেশকে উত্সাহিত করে যেখানে বিভিন্ন পটভূমি থেকে ব্যক্তিদের অবদানকে মূল্যায়ন এবং উদযাপন করা হয়।

- **অন্তর্ভুক্তি:** আমাদের দল এবং বৃহত্তর সম্প্রদায়ের মধ্যে বিভিন্ন দৃষ্টিভঙ্গি এবং পটভূমি গ্রহণ করা।
- **যোগাযোগ:** ধারণার বিনিময় এবং উদ্ভাবনী ফলাফলগুলি উত্সাহিত করতে সহকর্মীদের মধ্যে অংশীদারিত্ব উত্সাহিত করা।
- **স্বচ্ছতা:** আমাদের সম্প্রদায়ের সাথে উন্মুক্ত আলোচনা এবং উন্নয়ন ভাগাভাগির প্রতি প্রতিশ্রুতিবদ্ধ।

## ক্যারিয়ার ও সুযোগ
আমরা ক্রমাগত প্রতিভাবান ব্যক্তিদের খুঁজছি আমাদের দলে যোগ দিতে এবং আমাদের লক্ষ্যতে অবদান রাখতে। আপনি যদি একজন অভিজ্ঞ সফটওয়্যার প্রকৌশলী, ডেটা বিজ্ঞানী, গবেষক, বা সম্প্রদায় ব্যবস্থাপক হন, তাহলে Hugging Face-এ আপনার ক্যারিয়ার বৃদ্ধির জন্য অসংখ্য সুযোগ রয়েছে:

- একটি গতিশীল এবং অন্তর্ভুক্ত পরিবেশে কাজ করুন।
- প্রকল্পগুলিতে অংশগ্রহণ করুন যা AI এবং ML এর সীমানা বিস্তৃত করে।
- শিল্পের শীর্ষ বিশেষজ্ঞদের সাথে সহযোগিতা করুন এবং উদ্ভাবনী গবেষণায় অবদান রাখুন।

### উপলব্ধ পদসমূহ
বর্তমান খালি পদগুলি অন্বেষণ করতে আমাদের [Jobs page](https://huggingface.co/jobs) তে যান!

## আমাদের সাথে যোগ দিন
আপনি একটি এন্টারপ্রাইজ হতে পারেন AI সক্ষমতা ব্যবহার করতে চাইলেন, একটি ডেভেলপার হতে পারেন যারা Cutting-edge টুলগুলি অন্বেষণ করতে চায়, অথবা একটি সম্ভাব্য নিয়োগকর্তা যিনি একটি বিপ্লবী দলে যোগ দিতে চান, **Hugging Face** আপনাকে স্বাগতম জানায়। আসুন একসাথে কৃত্রিম বুদ্ধিমত্তার ভবিষ্যত গড়ে তুলি!

অতিরিক্ত তথ্যের জন্য, আমাদের ওয়েবসাইটে যান: [huggingface.co](https://huggingface.co)  
আপডেট এবং সম্প্রদায়ের সঙ্গে যোগাযোগের জন্য আমাদের সোশ্যাল মিডিয়াতে অনুসরণ করুন:  
- [GitHub](https://github.com/huggingface)  
- [Twitter](https://twitter.com/huggingface)  
- [LinkedIn](https://www.linkedin.com/company/huggingface)  
- [Discord](https://discord.gg/huggingface)  

আসুন একসাথে AI এর ভবিষ্যতকে গড়ে তুলি!

'# Hugging Face কোম্পানির ব্রোশার\n\n## সাধারণ তথ্য\n**Hugging Face** কৃত্রিম বুদ্ধিমত্তা (AI) এবং মেশিন লার্নিং (ML) বিপ্লবের অগ্রভাগে রয়েছে, একটি সহযোগিতামূলক প্ল্যাটফর্ম প্রদান করে যেখানে ডেভেলপার এবং গবেষকরা একত্রে কাজ করতে, শেয়ার করতে এবং নতুনত্ব আনতে পারেন। কোম্পানিটি ML সম্পদের জন্য একটি কেন্দ্রীয় কেন্দ্র হিসেবে ব্যাপকভাবে পরিচিত, যেখানে ১ মিলিয়নেরও বেশি মডেল, ২৫০k ডেটাসেট এবং অসংখ্য AI অ্যাপ্লিকেশন রয়েছে। Hugging Face ML সম্প্রদায়ের জন্য একটি অন্তর্ভুক্তিমূলক এবং উন্মুক্ত পরিবেশ তৈরি করতে প্রতিশ্রুতিবদ্ধ, গবেষক, প্রকৌশলী এবং ডেটা উন্মাদদের জন্য নতুনত্ব এবং সহযোগিতা উত্সাহিত করে।\n\n## আমাদের লক্ষ্য\n*Hugging Face: AI সম্প্রদায় ভবিষ্যত গড়ছে* — আমাদের লক্ষ্য হল ML ক্ষেত্রে সহযোগিতা বৃদ্ধি করা এবং বিশেষ করে উন্নত মডেল এবং ডেটাসেটকে সকলের জন্য আরও সহজলভ্য করে তোলা, একক ডেভেলপার থেকে শীর্ষ এন্টারপ্রাইজ পর্যন্ত।\n\n## গ্রাহকরা\n**৫০,০০০ এরও বেশি সংগঠন** Hugging Face ব্যবহার করে, যার মধ্যে মাইক্রোসফ্ট, গুগল, এবং অ্যামাজনের মতো কিছু বৃহত্তম প্রযুক্তি নাম অন্তর্ভুক্ত রয়েছে। আমরা